# OPAN5505 Lab 3 Assignment

## Background

You've been engaged by the Federal Aviation Administration to perform an analysis on "strike" data. A "strike" refers to when an aircraft and an object within the air (typically wildlife) collide.

## Prepare your environment

You will need the `polars` package for this assignment. We'll also use `datetime` for date handling.

In [11]:
import polars as pl
strikes = pl.read_csv( "https://raw.githubusercontent.com/philhetzel/opan5510-class10/refs/heads/main/data/faa_strikes.txt", separator="\t")
strikes = strikes.with_columns(pl.col("Collision Date and Time").str.strptime(pl.Datetime))

## Load your dataset

Read the `faa_strikes.txt` dataset into Polars. Name the resulting DataFrame `strikes`.

## Question 1: Running total of strikes by day through 2013

You are interested in seeing the running total of strikes over time (`Collision Date and Time`) on a daily basis. Using the `strikes` dataframe, create a new column that removes the time information but keeps the date information from the `Collision Date and Time` column (call this new field `date`). Next, aggregate the `Number of Strikes` by day (call this new field `daily_strikes`). Sort the data in ascending order by `date`. Then, create the running total of `daily_strikes` (name the new field `strikes_cumulative`) and filter to only the records **up to and including** the `date` of 2013-12-31. Name the resulting data frame `running_total_strikes`.

*Please place a Python code block and code below. The output should be a DataFrame. You must use a window function to receive credit on this question.*

In [32]:
import polars as pl
import datetime

# Load dataset
strikes = pl.read_csv(
    "https://raw.githubusercontent.com/philhetzel/opan5510-class10/refs/heads/main/data/faa_strikes.txt",
    separator="\t"
)

# Ensure datetime parsing
strikes = strikes.with_columns(
    pl.col("Collision Date and Time").str.strptime(pl.Datetime)
)

# 1. Create date-only column (remove time)
strikes = strikes.with_columns(
    pl.col("Collision Date and Time").dt.date().alias("date")
)

# 2. Aggregate Number of Strikes by day
daily_strikes_df = (
    strikes
    .group_by("date")
    .agg(pl.col("Number of Strikes").sum().alias("daily_strikes"))
    .sort("date")
)

# 3. Running total using window function
running_total_strikes = (
    daily_strikes_df
    .with_columns(
        pl.col("daily_strikes")
          .cumsum()
          .alias("strikes_cumulative")
    )
    .filter(pl.col("date") <= datetime.date(2013, 12, 31))
)

running_total_strikes

AttributeError: 'Expr' object has no attribute 'cumsum'

## Question 2: States with the third highest financial cost

The FAA is interested in the financial cost of strikes for each `Origin State`. Using the `strikes` dataframe: first, sum `Cost: Total $` by `Origin State` (name the resulting column `damage`) then `rank` these states in order of which state had the most financial `damage` (name the new column with rank information, `ranking`). Next, filter to show the row with the third highest `ranking`. Use a window function to answer this question. Name the resulting dataframe `damage_state`.

*Please place a Python code block and code below. The output should be a DataFrame. You must use a window function to receive credit on this question.*

In [21]:
# Your code here
# Sum total cost by Origin State
damage_by_state = (
    strikes
    .group_by("Origin State")
    .agg(pl.col("Cost: Total $").sum().alias("damage"))
)

# Add ranking using a window function (1 = highest damage)
damage_state = (
    damage_by_state
    .with_columns(
        pl.col("damage")
          .rank("dense", descending=True)
          .alias("ranking")
    )
    .filter(pl.col("ranking") == 3)  # Third highest
)

damage_state

Origin State,damage,ranking
str,i64,u32
"""California""",29671432,3


## Question 3: What are the second costliest Species Groups for each Aircraft Type?

The FAA wants to know if some species groups are more dangerous to particular types of aircraft. Using the `strikes` dataframe: first, sum financial damage (`Cost: Total $`) information by `Aircraft: Type` and `Wildlife: Species Group` (name the new field `damage`). Rank the rows within each `Aircraft: Type` based on the greatest amount of `damage`; when performing the ranking function, name the new column `ranking`. Return the rows that represent the `Wildlife: Species Group` values that caused the **second most** financial `damage` to each `Aircraft: Type`. Name the resulting dataframe `type_species`.

*Please place a Python code block and code below. The output should be a DataFrame. You must use a ranking function to receive credit on this question.*

In [23]:
# Your code here
# Your code here

# Sum financial damage by Aircraft Type and Wildlife Species Group
damage_by_type_species = (
    strikes
    .group_by(["Aircraft: Type", "Wildlife: Species Group"])
    .agg(pl.col("Cost: Total $").sum().alias("damage"))
)

# Rank species groups within each aircraft type based on damage
type_species = (
    damage_by_type_species
    .with_columns(
        pl.col("damage")
          .rank("dense", descending=True)
          .over("Aircraft: Type")  # Partition by Aircraft Type
          .alias("ranking")
    )
    .filter(pl.col("ranking") == 2) # Filter for the second highest ranking
    .sort(["Aircraft: Type", "damage"], descending=[False, True]) # Optional: Sort for better readability
)

type_species

Aircraft: Type,Wildlife: Species Group,damage,ranking
str,str,i64,u32
"""Airplane""","""Kites, Hawks, Eagles""",48989287,2
"""Helicopter""","""Vultures""",116748,2
"""NA""","""Vultures""",56227,2


## Question 4: Which days had the greatest positive jump in strikes?

The FAA wants to investigate which days had the largest increase in strikes from the previous day. Using the `strikes` dataframe: sum the `Number of Strikes` measure by day (note: not by day and time). You can use a similar technique from Question 1 to create a new column from `Collision Date and Time` without time information and with date information (name this column `date`). Next, compute the previous day's strikes using a window function (name the new column `previous_day`), and calculate the difference of strikes between the current day and the previous day (name the new column `delta_strikes`). Sort the resulting data by `delta_strikes` in descending order to find which days had the highest increase of strikes from the previous day. Name the resulting dataframe `greatest_strike_increase`.

**Hint:** This code is more involved that other exercises, so I will explicitly name the steps to answer the question.

1. Start with the `strikes` data frame
2. Create a new column called `date` which removes the time information in the `Collision Date and Time` column
3. Sum the `Number of Strikes` by `date`, and name the new column `daily_strikes`
4. Sort by the `date` column in ascending order
5. Create a new column called `previous_day` to calculate what the `daily_strikes` were in the previous day
6. Create a new column called `delta_strikes` which subtracts `previous_day` from `daily_strikes`
7. Sort in descending order by `delta_strikes`

*Please place a Python code block and code below. The output should be a DataFrame. You must use a window function to receive credit on this question.*

In [31]:
# Your code here
# Create a date-only column from the Collision Date and Time
strikes_with_date = strikes.with_columns(
    pl.col("Collision Date and Time")
      .cast(pl.Date)
      .alias("date")
)

# Sum the Number of Strikes by date
daily_strikes_df = (
    strikes_with_date
    .group_by("date")
    .agg(pl.col("Number of Strikes").sum().alias("daily_strikes"))
    .sort("date") # Sort by date in ascending order
)

# Compute the difference from the previous day using a window function
greatest_strike_increase = (
    daily_strikes_df
    .with_columns(
        (pl.col("daily_strikes") - pl.col("daily_strikes").shift(1).over([])).alias("delta_strikes")
    )
    .sort("delta_strikes", descending=True)
)

greatest_strike_increase

ComputeError: at least one key is required in a group_by operation

## Question 5: Which single day had the greatest increase in strikes for each `Aircraft: Type`?

FAA was interested in the exercise from the last question but now wants to determine the largest delta between days for each `Aircraft: Type`. What are the days that had largest positive change in strikes for each `Aircraft: Type`? In order to answer this question, you will need to perform the exercise from Question 4, but this time include a grouping by `Aircraft: Type`. After grouping by `Aircraft: Type`, add a column called `ranking` and use a window function to determine the day with the largest increase in strikes from the previous day. Name the resulting data frame `greatest_strike_increase_type`

NOTE: Helicopters are not struck with a high frequency and will not show up in your analysis; this is fine.

In [28]:
# Your code here
# Your code here

# Create a date-only column from the Collision Date and Time and include Aircraft Type
strikes_with_date_type = strikes.with_columns([
    pl.col("Collision Date and Time")
      .cast(pl.Date)
      .alias("date"),
    pl.col("Aircraft: Type")
])

# Sum the Number of Strikes by date and Aircraft Type
daily_strikes_by_type = (
    strikes_with_date_type
    .group_by(["Aircraft: Type", "date"])
    .agg(pl.col("Number of Strikes").sum().alias("daily_strikes"))
    .sort(["Aircraft: Type", "date"]) # Sort by Aircraft Type and date
)

# Compute the previous day's strikes and calculate the difference using a partitioned window function
greatest_strike_increase_type = (
    daily_strikes_by_type
    .with_columns([
        pl.col("daily_strikes")
          .shift(1)
          .over("Aircraft: Type")  # Partition by Aircraft Type
          .alias("previous_day"),
        (pl.col("daily_strikes") - pl.col("daily_strikes").shift(1).over("Aircraft: Type")).alias("delta_strikes") # Calculate the difference within each partition
    ])
    .filter(pl.col("delta_strikes").is_not_null()) # Filter out the first day of each partition where delta_strikes is null
    .with_columns(
        pl.col("delta_strikes")
          .rank("dense", descending=True)
          .over("Aircraft: Type") # Rank within each Aircraft Type based on delta_strikes
          .alias("ranking")
    )
    .filter(pl.col("ranking") == 1) # Filter for the day with the largest increase (ranking 1)
    .sort(["Aircraft: Type", "delta_strikes"], descending=[False, True]) # Optional: Sort for better readability
)

greatest_strike_increase_type

Aircraft: Type,date,daily_strikes,previous_day,delta_strikes,ranking
str,date,i64,i64,i64,u32
"""Airplane""",2014-08-25,25,8,17,1
"""Helicopter""",2010-07-21,2,1,1,1
"""Helicopter""",2012-11-02,2,1,1,1
"""Helicopter""",2013-10-28,2,1,1,1
"""Helicopter""",2014-08-14,2,1,1,1
"""NA""",2014-08-04,5,1,4,1


# Task
Analyze the `faa_strikes.txt` dataset to determine the state with the third highest total cost of strikes.

## Upload data

### Subtask:
Upload the `faa_strikes.txt` file using the file upload widget.


**Reasoning**:
The previous attempt to upload the file failed. I will try again to execute the code cell to upload the `faa_strikes.txt` file using the file upload widget.

